In [1]:
import pandas as pd
from TemporalAbstraction import CategoricalAbstraction
from sklearn.cluster import KMeans
import numpy as np

In [9]:
data = pd.read_csv(r'C:\Users\orzma\OneDrive\文件\MSc Compter Science\課程\碩一\ML for Quantified Self\ML4QS_Group125\data_w_features\data_w_all_features_0.5.csv')


In [10]:
# Identify columns of interest
columns_of_interest = [
    'amplitude_mean_kalman', 'pitch_mean_kalman', 'sound_intensity_mean_kalman']

In [12]:
# Function to categorize a feature using k-means clustering
def categorize_feature(data, n_clusters=3):
    # Identify columns of interest
    columns_of_interest = ['amplitude_mean_kalman', 'pitch_mean_kalman', 'sound_intensity_mean_kalman']
    for feature in columns_of_interest:
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        data[f'{feature}_categorical'] = kmeans.fit_predict(data[[feature]])
        
        # Assign meaningful category names
        centroids = kmeans.cluster_centers_.flatten()
        sorted_centroids = np.argsort(centroids)
        category_names = ['low', 'medium', 'high']
        mapping = {sorted_centroids[i]: category_names[i] for i in range(n_clusters)}
        
        data[f'{feature}_categorical'] = data[f'{feature}_categorical'].map(mapping)
    return data


In [13]:
# Group the data by language, tone, participant, and script
grouped_data = data.groupby(['language', 'tone', 'participant', 'script'])

# Define a function to process each group
def process_group(group):
    # Remove the first 10 rows of the group
    group = group.iloc[10:]
    return group

# Apply the process_group function to each group and reset the index
processed_data = grouped_data.apply(categorize_feature).reset_index(drop=True)

# Merge the processed data back into the original data
data = pd.concat([data[['language', 'tone', 'participant', 'script']], processed_data], axis=1)


c:\Users\orzma\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\orzma\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
c:\Users\orzma\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\orzma\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environme

In [14]:

# Display the number of each category for each column
for feature in columns_of_interest:
    print(data.groupby([f'{feature}_categorical']).size())  

amplitude_mean_kalman_categorical
high      5739
low       2421
medium    5979
dtype: int64
pitch_mean_kalman_categorical
high       911
low       7811
medium    5417
dtype: int64
sound_intensity_mean_kalman_categorical
high      6951
low       1568
medium    5620
dtype: int64


In [15]:
# Print out the column names to verify
print(data.columns.tolist())

['language', 'tone', 'participant', 'script', 'time_0.5', 'language', 'tone', 'participant', 'script', 'sound_intensity_max', 'sound_intensity_min', 'sound_intensity_mean', 'sound_intensity_median', 'sound_intensity_std', 'sound_intensity_missing', 'pitch_max', 'pitch_min', 'pitch_mean', 'pitch_median', 'pitch_std', 'pitch_missing', 'amplitude_mean', 'amplitude_max', 'amplitude_min', 'amplitude_median', 'amplitude_std', 'amplitude_missing', 'amplitude_mean_lowpass', 'pitch_mean_lowpass', 'sound_intensity_mean_lowpass', 'amplitude_mean_kalman', 'pitch_mean_kalman', 'sound_intensity_mean_kalman', 'sound_intensity_mean_kalman_max_freq', 'sound_intensity_mean_kalman_freq_weighted', 'sound_intensity_mean_kalman_pse', 'sound_intensity_mean_kalman_freq_0.0_Hz_ws_10', 'sound_intensity_mean_kalman_freq_0.1_Hz_ws_10', 'sound_intensity_mean_kalman_freq_0.2_Hz_ws_10', 'sound_intensity_mean_kalman_freq_0.3_Hz_ws_10', 'sound_intensity_mean_kalman_freq_0.4_Hz_ws_10', 'sound_intensity_mean_kalman_freq

In [19]:
data.to_csv('/Users/macbookair/Desktop/ML4QS_Group125/data_w_features/data_w_all_features_on_0.5_secs_with_categorical.csv', index=False)

In [20]:
# One-hot encode the categorical columns
categorical_cols = [f'{feature}_categorical' for feature in columns_of_interest]
data = pd.get_dummies(data, columns=categorical_cols, prefix=categorical_cols)



In [21]:
# Define parameters for the CategoricalAbstraction
cols = [col for col in data.columns if 'categorical' in col]
match = ['exact' for _ in cols]
min_support = 0.1
window_size = 3
max_pattern_size = 3

# Instantiate the CategoricalAbstraction class
ca = CategoricalAbstraction()

# Perform the temporal abstraction on the categorical data
abstracted_data = ca.abstract_categorical(data, cols, match, min_support, window_size, max_pattern_size)

 

# Save the resulting dataset with abstracted patterns
output_path = '/Users/macbookair/Desktop/ML4QS_Group125/data_w_features/data_w_temporal_patterns.csv'
abstracted_data.to_csv(output_path, index=False)

abstracted_data.head()


amplitude_mean_kalman_categorical_high
amplitude_mean_kalman_categorical_low
amplitude_mean_kalman_categorical_medium
pitch_mean_kalman_categorical_low
pitch_mean_kalman_categorical_medium
sound_intensity_mean_kalman_categorical_high
sound_intensity_mean_kalman_categorical_low
sound_intensity_mean_kalman_categorical_medium
Number of patterns of size 1 is 8
amplitude_mean_kalman_categorical_high(b)amplitude_mean_kalman_categorical_high
amplitude_mean_kalman_categorical_high(b)amplitude_mean_kalman_categorical_medium
amplitude_mean_kalman_categorical_high(b)pitch_mean_kalman_categorical_low
amplitude_mean_kalman_categorical_high(b)pitch_mean_kalman_categorical_medium
amplitude_mean_kalman_categorical_high(c)pitch_mean_kalman_categorical_medium
amplitude_mean_kalman_categorical_high(b)sound_intensity_mean_kalman_categorical_high
amplitude_mean_kalman_categorical_high(c)sound_intensity_mean_kalman_categorical_high
amplitude_mean_kalman_categorical_high(b)sound_intensity_mean_kalman_categor

,time_0.5,language,tone,participant,script,sound_intensity_max,sound_intensity_min,sound_intensity_mean,sound_intensity_median,sound_intensity_std,...,temp_pattern_sound_intensity_mean_kalman_categorical_medium(b)amplitude_mean_kalman_categorical_low,temp_pattern_sound_intensity_mean_kalman_categorical_medium(b)amplitude_mean_kalman_categorical_medium,temp_pattern_sound_intensity_mean_kalman_categorical_medium(c)amplitude_mean_kalman_categorical_medium,temp_pattern_sound_intensity_mean_kalman_categorical_medium(b)pitch_mean_kalman_categorical_low,temp_pattern_sound_intensity_mean_kalman_categorical_medium(c)pitch_mean_kalman_categorical_low,temp_pattern_sound_intensity_mean_kalman_categorical_medium(b)pitch_mean_kalman_categorical_medium,temp_pattern_sound_intensity_mean_kalman_categorical_medium(c)pitch_mean_kalman_categorical_medium,temp_pattern_sound_intensity_mean_kalman_categorical_medium(b)sound_intensity_mean_kalman_categorical_high,temp_pattern_sound_intensity_mean_kalman_categorical_medium(b)sound_intensity_mean_kalman_categorical_low,temp_pattern_sound_intensity_mean_kalman_categorical_medium(b)sound_intensity_mean_kalman_categorical_medium
0,0.0,ch,bus,subject1,t1,25.877110,20.899051,23.459924,23.512067,1.417073,...,0,0,0,0,0,0,0,0,0,0
1,0.5,ch,bus,subject1,t1,23.891661,17.708520,20.647569,20.433223,2.398310,...,0,0,0,0,0,0,0,0,0,0
2,1.0,ch,bus,subject1,t1,43.304747,20.569049,26.230642,23.424227,7.978059,...,0,0,0,0,0,0,0,0,0,0
3,1.5,ch,bus,subject1,t1,44.027427,21.335389,33.193647,33.533856,8.111167,...,0,0,0,0,0,0,0,0,0,0
4,2.0,ch,bus,subject1,t1,27.134905,19.190781,22.226163,22.095150,2.726114,...,0,0,0,0,0,0,0,0,0,0
